In [ ]:
%pip install "pydantic-ai[logfire]" "nest_asyncio"

In [ ]:
from agents import Agent, Runner

import logfire
# configure Logfire for console only
logfire.configure(
    service_name="my_agent_service",
    send_to_logfire=False,
    inspect_arguments=True
)
logfire.instrument_openai_agents()

agent = Agent(
    name="Research Assistant",
    instructions="You are a helpful Research Assistant which helps professors in various tasks.",
)

result = await Runner.run(agent, "Get me the abstract of arXiv:1706.03762")
print(result.final_output)

In [ ]:
import requests
from bs4 import BeautifulSoup
from typing import Dict, Any
from agents import function_tool


@function_tool
def fetch_paper(arxiv_id: str) -> Dict[str, Any]:
    """
    Fetches paper information from arXiv given its ID.
    
    Args:
        arxiv_id: The arXiv ID (e.g., '1706.03762')
    
    Returns:
        A dictionary containing paper information including:
        - title
        - authors
        - abstract
        - pdf_url
        - published_date
    """
    # Clean the input ID
    arxiv_id = arxiv_id.strip().replace("arXiv:", "")
    
    # Construct the URL
    url = f"https://arxiv.org/abs/{arxiv_id}"
    
    try:
        # Fetch the page
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract paper information
        title = soup.select_one("h1.title").text.replace("Title:", "").strip()
        authors = [author.strip() for author in soup.select_one("div.authors").text.replace("Authors:", "").split(",")]
        abstract = soup.select_one("blockquote.abstract").text.replace("Abstract:", "").strip()
        pdf_url = f"https://arxiv.org/pdf/{arxiv_id}.pdf"
        
        # Try to extract published date
        date_element = soup.select_one("div.dateline")
        published_date = date_element.text.strip() if date_element else "Date not found"
        
        return {
            "title": title,
            "authors": authors,
            "abstract": abstract,
            "pdf_url": pdf_url,
            "published_date": published_date,
            "arxiv_id": arxiv_id
        }
        
    except Exception as e:
        print(f"Error fetching paper: {e}")
        return {
            "error": str(e),
            "arxiv_id": arxiv_id
        }


agent_1 = Agent(
    name="Research Assistant",
    instructions="You are a helpful Research Assistant which helps professors in various tasks.",
    tools=[fetch_paper]
)

result = await Runner.run(agent_1, "Get me the abstract of arXiv:1706.03762")
print(result.final_output)

In [ ]:
result = await Runner.run(agent_1, "Get me the abstract of arXiv:1706.03762 and tell me where each author is working currently")
print(result.final_output)

In [ ]:
import logging
from agents import WebSearchTool


# Set logging level to debug
logging.basicConfig(level=logging.DEBUG)

agent_2 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper],
)

result = await Runner.run(
    agent_2,
    "Get me the abstract of arXiv:1706.03762 and tell me where each author is working currently.")
print(result.final_output)

In [ ]:

from agents import WebSearchTool


agent_2 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper],
)

result = await Runner.run(
    agent_2,
    "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic")
print(result.final_output)

In [ ]:
from typing import Annotated
from agents import function_tool


@function_tool
async def read_local_file(
    file_path: Annotated[str, "Path to the file to read"]
) -> str:
    """
    Reads the content of a local file and returns it as a string.
    
    Args:
        file_path: Path to the file to read
        
    Returns:
        The content of the file as a string
        
    Raises:
        FileNotFoundError: If the file does not exist
        PermissionError: If the file cannot be read due to permission issues
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except PermissionError:
        return f"Error: Permission denied when trying to read '{file_path}'."
    except Exception as e:
        return f"Error reading file: {str(e)}"


agent_3 = Agent(
    name="Research Assistant",
    instructions=
    """
You are a helpful Research Assistant which helps professors in various tasks.
Make sure to answer all questions asked. When asked about current information,
use the WebSearchTool to find the latest information.
""",
    tools=[WebSearchTool(), fetch_paper, read_local_file],
)

result = await Runner.run(
    agent_3,
    "check my draft proposal at draft_proposal.txt and give me feedback on it based on what is currently available in the web about this topic")
print(result.final_output)